In [1]:
import pickle
import pandas as pd
from sklearn.metrics import *
import operator
import numpy as np
from scipy import stats

def bestGuess(array, devices):
    index = np.where(array == array.max())
    index = int(index[0][0])
    return devices[index]

def bestGuessIndex(array, devices):
    index = np.where(array == array.max())
    return int(index[0][0])

def zscore(table, confidence, idxDevice):
    x = confidence  
    mean = table['Avg Conf'][idxDevice]
    stdev = table['Std Conf'][idxDevice]
    z = (x - mean) / stdev

    return z
    
def gotItRight(predictions_proba, predictions, devices, y):
    correct = 0
    other = 0
    print(devices)
    print(len(predictions_proba) == len(y))
    for i in range(len(predictions)):
        bestGuess = predictions[i]
        if devices[bestGuess] == y[i]:
            correct+=1
        else:
        
            other+=1
    print("correct is " + str(correct))
    print(len(predictions))

def makePrediction(model, unseen, devices, table, labels):
    loadedModel = pickle.load(open(model, 'rb'))
    unseen = unseen.reindex(sorted(unseen.columns), axis=1)

    
    unseen.drop(unseen.columns[unseen.columns.str.contains('Unname',case = False)],axis = 1, inplace = True)
    
    y = unseen['Device'].tolist() #unseen should only 1 or more devices
    
    for col in labels:
        if col in unseen.columns:
            unseen = unseen.drop(columns= [col])
    print("length of unseen in columns is " + str(len(unseen.columns)))
    predictions = loadedModel.predict(unseen)
    
    
    predictions_proba = loadedModel.predict_proba(unseen)
    print(len(predictions_proba[0]))
    i = 0
    
    uncertainReplay = 0
    uncertainDevice = 0
    correctDevice = 0
    wrongDevice = 0
    certainReplay = 0
    incorrectDevice = 0
    replayCount = 0
    confusedSame = 0
    
    for i in range(0, len(predictions_proba)):
        if 'replay' in y[i].lower():
            replayCount +=1 
        myBestGuess = devices[predictions[i]]
        
        
        myBestGuessIndex = predictions[i] # Predictions contains the besrt guesses, myBestGuessIndex is the bestGuess for each reading
        
        confidence = predictions_proba[i].max()
        
        if zscore(table, confidence , myBestGuessIndex) < -3: # Very Uncertain
            
            
            if 'replay' in y[i].lower(): # Uncertain and its a replay
                uncertainReplay += 1
            else:
                print("truth " + y[i])
                print("guess " + myBestGuess + " " + str(confidence) )
                uncertainDevice += 1 # Uncertain and it's just a device
        else:
            
            if myBestGuess == y[i]:  # Certain and got it right
                correctDevice += 1
            elif myBestGuess[:-2] == y[i][:-2]:
                confusedSame += 1
            else: 
                if 'replay' in y[i].lower(): # Certain and it was a replay 
                    print("Cetrain " + y[i].lower())
                    certainReplay += 1
                else:
                    
                    incorrectDevice += 1
                    
                wrongDevice += 1
    gotItRight(predictions_proba, predictions, devices, y)
    print("I made " + str(len(predictions)) + " predictions. I was uncertain at a replay (Very Good)" 
        + str(uncertainReplay) + " out of " + str(replayCount)  + ". I was uncertain at a genuine device (Bad) " 
        + str(uncertainDevice) + ". I was certain at a Replay (Very Bad)" + str(certainReplay) 
        + ". I was certain at the right device (Very Good)" + str(correctDevice) 
        + " I was certain but the wrong device (Very Bad)" + str(incorrectDevice))
    
if __name__ == '__main__':
    testDir = r'C:\Users\brnma\test.xlsx'
    path = r'C:\Users\brnma\train.xlsx'
    clf = 'RandomForest d=15 n_est=15Model.sav'
    test = pd.read_excel(testDir)
    df = pd.read_excel(path)
    devices =  df['Device'].unique()
    print(len(devices))
    confidence = pd.read_excel('smallerTable.xlsx')
    labels = ['Device','Model','App','Distance']
    
    makePrediction(clf, test, devices, confidence, labels)
    print("Done")
    

17
length of unseen in columns is 49
14
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
Cetrain replay(b)
truth iHealth Blood Pressure(A)
guess iHealth Blood Pressure(A) 0.5714285714285714
truth Portable ECG moniter(A)
guess Gian Iphone(A) 0.5
truth Portable ECG moniter(A)
guess Gian Iphone(A) 0.42857142857142855
truth Portable ECG moniter(A)
g